### 서울 공원 분석

In [1]:
import json, folium
import numpy as np
import pandas as pd

In [2]:
park = pd.read_csv('data/서울공원요약.csv')
park.head(3)

,공원명,면적,주소,경도,위도,분류,size,크기
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,대형,896,15
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,소형,896,3
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449,대형,896,15


In [3]:
park['자치구'] = park.주소.apply(lambda x: x.split()[1])
park.자치구.unique()

array(['중구', '강동구', '과천시', '성동구', '마포구', '종로구', '영등포구', '강서구', '동대문구',
       '동작구', '광진구', '용산구', '서대문구', '서초구', '관악구', '금천구', '성북구', '구로구',
       '양천구', '강남구', '송파구', '도봉구', '강북구', '은평구', '중랑구', '노원구', '공릉2동'],
      dtype=object)

In [4]:
# 잘못된 데이터중 공릉2동은 노원구로 변경
park.자치구.replace('공릉2동','노원구', inplace=True)
park.자치구.unique()

array(['중구', '강동구', '과천시', '성동구', '마포구', '종로구', '영등포구', '강서구', '동대문구',
       '동작구', '광진구', '용산구', '서대문구', '서초구', '관악구', '금천구', '성북구', '구로구',
       '양천구', '강남구', '송파구', '도봉구', '강북구', '은평구', '중랑구', '노원구'],
      dtype=object)

In [5]:
df = park.pivot_table('면적', '자치구', aggfunc='sum')
df.head()

,면적
자치구,
강남구,3626398
강동구,1526387
강북구,2047054
강서구,2194152
과천시,9132690


In [6]:
df.drop(['과천시'], inplace=True)
df.head()

,면적
자치구,
강남구,3626398
강동구,1526387
강북구,2047054
강서구,2194152
관악구,10431453


In [7]:
pop = pd.read_csv('../02.Pandas/data/pop.csv', index_col='구별')
pop.head()

,인구수,내국인,외국인,고령자,외국인비율,고령자비율
구별,,,,,,
종로구,153789,144683,9106,27818,5.92,18.09
중구,131787,122499,9288,24392,7.05,18.51
용산구,237285,222953,14332,39070,6.04,16.47
성동구,292672,285990,6682,46380,2.28,15.85
광진구,352627,339996,12631,51723,3.58,14.67


In [8]:
df = df.merge(pop, left_index=True, right_index=True)
df.head()

,면적,인구수,내국인,외국인,고령자,외국인비율,고령자비율
강남구,3626398,537800,533042,4758,78226,0.88,14.55
강동구,1526387,466472,462664,3808,74070,0.82,15.88
강북구,2047054,302563,299182,3381,64333,1.12,21.26
강서구,2194152,579768,574315,5453,92558,0.94,15.96
관악구,10431453,499449,485699,13750,79871,2.75,15.99


In [9]:
df['인당 면적'] = (df.면적 / df.인구수).round(2)
df = df[['면적','인구수','인당 면적']]
df.head()

,면적,인구수,인당 면적
강남구,3626398,537800,6.74
강동구,1526387,466472,3.27
강북구,2047054,302563,6.77
강서구,2194152,579768,3.78
관악구,10431453,499449,20.89


In [10]:
fname = 'data/seoul_geo_simple.json'
geo_data = json.load(open(fname, encoding='utf-8'))

In [11]:
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['id']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

gu_dict = get_text_location(geo_data)

- 자치구별 공원 면적

In [12]:
map = folium.Map([37.55, 126.98], zoom_start=10, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df.면적,       # 단계구분도로 보여줄 데이터
    columns=[df.index, df.면적],    # 데이터프레임에서 추출할 항목
    fill_color='YlOrRd',      # Colormap
    key_on='feature.id'     # 지도에서 조인할 항목
).add_to(map)
for gu_name in df.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울시 자치구별 공원 면적</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

- 자치구별 인당 공원면적

In [13]:
map = folium.Map([37.55, 126.98], zoom_start=10, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df['인당 면적'],       # 단계구분도로 보여줄 데이터
    columns=[df.index, df['인당 면적']],    # 데이터프레임에서 추출할 항목
    fill_color='GnBu',      # Colormap
    key_on='feature.id'     # 지도에서 조인할 항목
).add_to(map)
for gu_name in df.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울시 자치구별 인당 공원면적</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map